# Mapping company reviews with competency

We are trying to gain insight into the work values essential for a certain company for a certain job role apart.
This activity can help Employer understand the factors that can attract potential candidate

# Loading Libraries

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import spacy
from textblob import TextBlob

# Downloading Pre-trained Spacy Model 

In [2]:
# Download pretrained english model 
try:
    import en_core_web_lg
except:
    !python -m spacy download en_core_web_lg
    import en_core_web_lg

     |████████▎                       | 214.6 MB 8.2 MB/s eta 0:01:161  |                                | 1.1 MB 587 kB/s eta 0:23:28     |                                | 2.6 MB 587 kB/s eta 0:23:26     |▏                               | 3.9 MB 587 kB/s eta 0:23:24     |▏                               | 4.7 MB 587 kB/s eta 0:23:22     |▎                               | 6.4 MB 587 kB/s eta 0:23:19     |▎                               | 6.8 MB 587 kB/s eta 0:23:19     |▍                               | 9.4 MB 9.5 MB/s eta 0:01:26     |▍                               | 10.9 MB 9.5 MB/s eta 0:01:26     |▌                               | 12.0 MB 9.5 MB/s eta 0:01:26     |▌                               | 12.3 MB 9.5 MB/s eta 0:01:26     |▌                               | 12.9 MB 9.5 MB/s eta 0:01:26     |▌                               | 14.4 MB 9.5 MB/s eta 0:01:26     |▋                               | 15.1 MB 9.5 MB/s eta 0:01:26     |▋                               | 15.8 MB 9.5 MB/s

     |███████████████████████▍        | 603.9 MB 6.6 MB/s eta 0:00:349    |██████████████▊                 | 379.5 MB 2.9 MB/s eta 0:02:34     |██████████████▉                 | 382.8 MB 2.9 MB/s eta 0:02:33     |██████████████▉                 | 384.1 MB 2.9 MB/s eta 0:02:32     |███████████████                 | 385.6 MB 2.9 MB/s eta 0:02:32     |███████████████                 | 387.4 MB 2.9 MB/s eta 0:02:31     |███████████████                 | 387.9 MB 10.4 MB/s eta 0:00:43     |███████████████                 | 389.4 MB 10.4 MB/s eta 0:00:43     |███████████████▏                | 392.5 MB 10.4 MB/s eta 0:00:43     |███████████████▏                | 393.3 MB 10.4 MB/s eta 0:00:42     |███████████████▏                | 393.3 MB 10.4 MB/s eta 0:00:42     |███████████████▎                | 394.8 MB 10.4 MB/s eta 0:00:42     |███████████████▎                | 395.1 MB 10.4 MB/s eta 0:00:42     |███████████████▍                | 398.0 MB 10.4 MB/s eta 0:00:42     |███████████████▍    

     |███████████████████████▍        | 605.3 MB 6.6 MB/s eta 0:00:34

     |████████████████████████████████| 827.9 MB 3.2 MB/s eta 0:00:012    |███████████████████████▌        | 607.0 MB 6.6 MB/s eta 0:00:34     |███████████████████████▌        | 608.5 MB 6.6 MB/s eta 0:00:34     |███████████████████████▌        | 608.9 MB 6.6 MB/s eta 0:00:34     |███████████████████████▋        | 611.5 MB 6.6 MB/s eta 0:00:33     |███████████████████████▊        | 612.5 MB 6.6 MB/s eta 0:00:33     |███████████████████████▊        | 613.6 MB 6.6 MB/s eta 0:00:33     |████████████████████████▏       | 623.8 MB 4.4 MB/s eta 0:00:47     |████████████████████████▏       | 625.2 MB 4.4 MB/s eta 0:00:47     |████████████████████████▏       | 625.9 MB 4.4 MB/s eta 0:00:46     |████████████████████████▎       | 626.8 MB 2.5 MB/s eta 0:01:22     |████████████████████████▍       | 629.9 MB 2.5 MB/s eta 0:01:20     |████████████████████████▌       | 633.6 MB 2.5 MB/s eta 0:01:19     |████████████████████████▌       | 634.3 MB 2.5 MB/s eta 0:01:19     |████████████████████████▌   

  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180943 sha256=3575e1a3d3e984fac2a73a95d130170411fb65185d6448bc134ead3939e3679c
  Stored in directory: /tmp/pip-ephem-wheel-cache-kib79qqr/wheels/02/24/f6/f88cfe42070c310a2842004556e8ca72637e5cb447e375b925
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


# Loading Data set from SQLite Db

In [3]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("collectors/data.sqlite3")
reviews_df = pd.read_sql_query("SELECT * from company_review", con)
con.close()

In [4]:
# Verify that result of SQL query is stored in the dataframe
reviews_df.head()

,id,company,title,rating,author,author_status,location,date,description,source
0,1,bmo financial group,Decent,3.0,Credit Analyst (Current Employee),N/A,"Burnaby, BC",11 July 2018,Not a bad job but can get boring easily. Manag...,indeed.com
1,2,bmo financial group,easy place to work,3.0,Project Manager (Former Employee),N/A,"Toronto, ON",7 April 2020,co-workers are friendly but pay is not good at...,indeed.com
2,3,bmo financial group,Dead Industry- They will squeeze you like a le...,1.0,Assistant Manager (Former Employee),N/A,"Cambridge, ON",6 April 2020,They give you the illusion of how great of a j...,indeed.com
3,4,bmo financial group,"In BMO agile team, one developer has to servic...",2.0,Software Specialist (Former Employee),N/A,"Toronto, ON",6 April 2020,"Basically, in BMO agile team, they hire 5+ per...",indeed.com
4,5,bmo financial group,Good,5.0,Relationship Manager (Current Employee),N/A,"Nanaimo, BC",4 April 2020,Work/life balance and customer centric. Traini...,indeed.com


# Loading Compentencies of O*Net DB

In [5]:
onet_competencies = 'datasets/competencies.csv'
onet_df = pd.read_csv(onet_competencies,index_col=0)
onet_df[onet_df['occupation'] == 'Computer and Information Research Scientists'].head(5)

,occupation,competency,category,description
0,Computer and Information Research Scientists,Source code management SCM software,Technology Skills,Development environment software
1,Computer and Information Research Scientists,Microsoft Azure,Technology Skills,Development environment software
2,Computer and Information Research Scientists,Visualization,Abilities,The ability to imagine how something will look...
3,Computer and Information Research Scientists,Free-field speakers,Tools Used,Loudspeakers
4,Computer and Information Research Scientists,Data visualization software,Technology Skills,Analytical or scientific software


# Data Preprocessing

In [6]:
def process_text(text):
    doc = nlp(text)
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            continue
        # result.append(token.lemma_)
        result.append(token.text)
    return " ".join(result)

In [7]:
nlp = en_core_web_lg.load()
nlp.Defaults.stop_words.add("\n")

In [8]:
onet_comp = np.array(onet_df['competency'])
onet_desc = np.array(onet_df['description'])

In [9]:
onet_comp = np.array([process_text(comp.lower()) for comp in onet_comp])
onet_desc = np.array([process_text(desc.lower()) for desc in onet_desc])

# Data Filtering

In [37]:
# Filter ONet DF to keep only required category of competencies

category_list = ['Work Values','Work Context']

print("ONET length before filter: ", len(onet_df))
filtered_onet_df = onet_df.loc[onet_df['category'].isin(category_list)]
print("ONET length after filter:  ", len(filtered_onet_df))
filtered_onet_df.head(5)

ONET length before filter:  4938
ONET length after filter:   116


,occupation,competency,category,description
44,Computer and Information Research Scientists,Relationships,Work Values,Occupations that satisfy this work value allow...
45,Computer and Information Research Scientists,Independence,Work Values,Occupations that satisfy this work value allow...
68,Computer and Information Research Scientists,Achievement,Work Values,Occupations that satisfy this work value are r...
106,Computer and Information Research Scientists,Support,Work Values,Occupations that satisfy this work value offer...
158,Computer and Information Research Scientists,Working Conditions,Work Values,Occupations that satisfy this work value offer...


# Loading Occupation DataSet

In [38]:
occupations = 'results/title_occupation.csv'
occupations = pd.read_csv(occupations)
occupations['description'] = reviews_df['description']
occupations

,id,title,identifier,score_all,name,description
0,1,Data Scientist,15-1111.00,0.883895,Computer and Information Research Scientists,Not a bad job but can get boring easily. Manag...
1,2,Business Intelligence Analyst,15-1199.08,0.866399,Business Intelligence Analysts,co-workers are friendly but pay is not good at...
2,3,Human Resources Data Scientist,15-1111.00,0.737225,Computer and Information Research Scientists,They give you the illusion of how great of a j...
3,4,Lead - Human Resource Data Scientist,15-1111.00,0.738833,Computer and Information Research Scientists,"Basically, in BMO agile team, they hire 5+ per..."
4,5,Machine Learning Engineer,15-1111.00,0.847708,Computer and Information Research Scientists,Work/life balance and customer centric. Traini...
...,...,...,...,...,...,...
1445,1456,Human Resources Data Scientist,15-1111.00,0.737225,Computer and Information Research Scientists,NaN
1446,1457,Lead - Human Resource Data Scientist,15-1111.00,0.738833,Computer and Information Research Scientists,NaN
1447,1458,Principal Data Engineer,15-1199.07,0.867460,Data Warehousing Specialists,NaN
1448,1459,Data Engineer,15-1199.07,0.846451,Data Warehousing Specialists,NaN


In [39]:
reviews_desc = occupations['description'].values
print(len(reviews_desc))

1450


# Data Refactoring

In [40]:
desc_lengths = np.array([len(list(nlp(str(desc)).sents)) for desc in reviews_desc])

In [41]:
desc_lengths[:15]

array([4, 5, 7, 6, 4, 8, 3, 1, 2, 5, 3, 4, 3, 2, 5])

In [42]:
# Choose to keep jobs which have sentences less than a threshold
MAX_SENTENCE_LENGTH = 40
job_ids = np.where(desc_lengths < MAX_SENTENCE_LENGTH)[0]

In [43]:
job_ids[:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [44]:
FIRST_N = 500

small_reviews_desc = reviews_desc[job_ids][:FIRST_N]

small_reviews_desc = np.array([np.array([process_text(line.text).replace("\n", '') \
                                for line in nlp(str(desc)).sents]) \
                                for desc in small_reviews_desc])

In [45]:
small_reviews_desc[0][:5]

array(['Not bad job boring easily', 'Management nice laid culture',
       'Focus sales policy', 'Not growth opportunities'], dtype='<U28')

In [46]:
small_reviews_desc.shape

(500,)

# Data Processing 

In [47]:
# ***************************************************************
# BEWARE: EXTREMELY TIME CONSUMING
# TRY FOR SMALL NUMBER OF REVIEWS
# IMPRACTICAL FOR A LARGE SET OF JOBS SIMULTANEOUSLY
# PRACTICAL IMPLICATION: PERFORM THIS WHENEVER A NEW COMPANY REVIEW IS ADDED
# ***************************************************************

scores = []
comp_ids = []

for i, desc in enumerate(small_reviews_desc):
    title = occupations['name'].iloc[i]
    idss = filtered_onet_df.index[filtered_onet_df['occupation'] == title].tolist()
    sent_score = []
    sent_comp_id = []
    print("COMPETENCIES {}, SENTENCES {}".format(len(idss), len(desc)))
    print("------------------------------")
    for j, sentence in enumerate(desc):
        sentence = nlp(str(sentence))
        temp_score = []
        # temp_comp_id = np.empty()
        for ids in idss:
            sim1 = sentence.similarity(nlp(str(onet_comp[ids])))
            sim2 = sentence.similarity(nlp(str(onet_desc[ids])))
            temp_score.append(max(sim1, sim2))
    
        max_score = np.max(temp_score)
        max_comp_id = idss[np.argmax(temp_score)]
        sent_score.append(max_score)
        sent_comp_id.append(max_comp_id)
        print("JOB {}, SENTENCE {} DONE".format(i, j))

    scores.append(sent_score)
    comp_ids.append(sent_comp_id)

COMPETENCIES 6, SENTENCES 4
------------------------------
JOB 0, SENTENCE 0 DONE
JOB 0, SENTENCE 1 DONE
JOB 0, SENTENCE 2 DONE
JOB 0, SENTENCE 3 DONE
COMPETENCIES 6, SENTENCES 5
------------------------------
JOB 1, SENTENCE 0 DONE
JOB 1, SENTENCE 1 DONE
JOB 1, SENTENCE 2 DONE
JOB 1, SENTENCE 3 DONE
JOB 1, SENTENCE 4 DONE
COMPETENCIES 6, SENTENCES 7
------------------------------
JOB 2, SENTENCE 0 DONE
JOB 2, SENTENCE 1 DONE
JOB 2, SENTENCE 2 DONE
JOB 2, SENTENCE 3 DONE
JOB 2, SENTENCE 4 DONE
JOB 2, SENTENCE 5 DONE


/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__mai

JOB 2, SENTENCE 6 DONE
COMPETENCIES 6, SENTENCES 6
------------------------------
JOB 3, SENTENCE 0 DONE
JOB 3, SENTENCE 1 DONE
JOB 3, SENTENCE 2 DONE
JOB 3, SENTENCE 3 DONE
JOB 3, SENTENCE 4 DONE
JOB 3, SENTENCE 5 DONE
COMPETENCIES 6, SENTENCES 4
------------------------------
JOB 4, SENTENCE 0 DONE
JOB 4, SENTENCE 1 DONE
JOB 4, SENTENCE 2 DONE
JOB 4, SENTENCE 3 DONE
COMPETENCIES 6, SENTENCES 8
------------------------------
JOB 5, SENTENCE 0 DONE
JOB 5, SENTENCE 1 DONE
JOB 5, SENTENCE 2 DONE
JOB 5, SENTENCE 3 DONE
JOB 5, SENTENCE 4 DONE
JOB 5, SENTENCE 5 DONE
JOB 5, SENTENCE 6 DONE
JOB 5, SENTENCE 7 DONE
COMPETENCIES 6, SENTENCES 3
------------------------------
JOB 6, SENTENCE 0 DONE
JOB 6, SENTENCE 1 DONE
JOB 6, SENTENCE 2 DONE
COMPETENCIES 6, SENTENCES 1
------------------------------
JOB 7, SENTENCE 0 DONE
COMPETENCIES 6, SENTENCES 2
------------------------------
JOB 8, SENTENCE 0 DONE
JOB 8, SENTENCE 1 DONE
COMPETENCIES 6, SENTENCES 5
------------------------------
JOB 9, SENTE

/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__mai

JOB 11, SENTENCE 1 DONE
JOB 11, SENTENCE 2 DONE
JOB 11, SENTENCE 3 DONE
COMPETENCIES 6, SENTENCES 3
------------------------------
JOB 12, SENTENCE 0 DONE
JOB 12, SENTENCE 1 DONE
JOB 12, SENTENCE 2 DONE
COMPETENCIES 6, SENTENCES 2
------------------------------
JOB 13, SENTENCE 0 DONE
JOB 13, SENTENCE 1 DONE
COMPETENCIES 6, SENTENCES 5
------------------------------
JOB 14, SENTENCE 0 DONE
JOB 14, SENTENCE 1 DONE
JOB 14, SENTENCE 2 DONE
JOB 14, SENTENCE 3 DONE
JOB 14, SENTENCE 4 DONE
COMPETENCIES 6, SENTENCES 6
------------------------------
JOB 15, SENTENCE 0 DONE
JOB 15, SENTENCE 1 DONE
JOB 15, SENTENCE 2 DONE
JOB 15, SENTENCE 3 DONE
JOB 15, SENTENCE 4 DONE
JOB 15, SENTENCE 5 DONE
COMPETENCIES 6, SENTENCES 1
------------------------------
JOB 16, SENTENCE 0 DONE
COMPETENCIES 6, SENTENCES 4
------------------------------
JOB 17, SENTENCE 0 DONE
JOB 17, SENTENCE 1 DONE
JOB 17, SENTENCE 2 DONE
JOB 17, SENTENCE 3 DONE
COMPETENCIES 6, SENTENCES 3
------------------------------
JOB 18, SEN

...

/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__mai

JOB 395, SENTENCE 1 DONE
JOB 395, SENTENCE 2 DONE


/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__mai

JOB 395, SENTENCE 3 DONE
JOB 395, SENTENCE 4 DONE
COMPETENCIES 6, SENTENCES 3
------------------------------
JOB 396, SENTENCE 0 DONE
JOB 396, SENTENCE 1 DONE
JOB 396, SENTENCE 2 DONE
COMPETENCIES 6, SENTENCES 7
------------------------------
JOB 397, SENTENCE 0 DONE
JOB 397, SENTENCE 1 DONE
JOB 397, SENTENCE 2 DONE
JOB 397, SENTENCE 3 DONE
JOB 397, SENTENCE 4 DONE
JOB 397, SENTENCE 5 DONE
JOB 397, SENTENCE 6 DONE
COMPETENCIES 6, SENTENCES 4
------------------------------
JOB 398, SENTENCE 0 DONE
JOB 398, SENTENCE 1 DONE
JOB 398, SENTENCE 2 DONE
JOB 398, SENTENCE 3 DONE
COMPETENCIES 6, SENTENCES 1
------------------------------
JOB 399, SENTENCE 0 DONE
COMPETENCIES 6, SENTENCES 4
------------------------------
JOB 400, SENTENCE 0 DONE
JOB 400, SENTENCE 1 DONE
JOB 400, SENTENCE 2 DONE
JOB 400, SENTENCE 3 DONE
COMPETENCIES 6, SENTENCES 3
------------------------------
JOB 401, SENTENCE 0 DONE
JOB 401, SENTENCE 1 DONE
JOB 401, SENTENCE 2 DONE
COMPETENCIES 6, SENTENCES 4
------------------

JOB 447, SENTENCE 1 DONE
JOB 447, SENTENCE 2 DONE
COMPETENCIES 6, SENTENCES 3
------------------------------
JOB 448, SENTENCE 0 DONE
JOB 448, SENTENCE 1 DONE
JOB 448, SENTENCE 2 DONE
COMPETENCIES 6, SENTENCES 7
------------------------------
JOB 449, SENTENCE 0 DONE
JOB 449, SENTENCE 1 DONE
JOB 449, SENTENCE 2 DONE
JOB 449, SENTENCE 3 DONE
JOB 449, SENTENCE 4 DONE
JOB 449, SENTENCE 5 DONE
JOB 449, SENTENCE 6 DONE
COMPETENCIES 6, SENTENCES 2
------------------------------
JOB 450, SENTENCE 0 DONE
JOB 450, SENTENCE 1 DONE
COMPETENCIES 6, SENTENCES 3
------------------------------
JOB 451, SENTENCE 0 DONE
JOB 451, SENTENCE 1 DONE
JOB 451, SENTENCE 2 DONE
COMPETENCIES 6, SENTENCES 5
------------------------------
JOB 452, SENTENCE 0 DONE
JOB 452, SENTENCE 1 DONE
JOB 452, SENTENCE 2 DONE
JOB 452, SENTENCE 3 DONE
JOB 452, SENTENCE 4 DONE
COMPETENCIES 6, SENTENCES 2
------------------------------
JOB 453, SENTENCE 0 DONE
JOB 453, SENTENCE 1 DONE
COMPETENCIES 6, SENTENCES 1
------------------

In [48]:
competency = []
for i in range(len(scores)):
    tup = list(zip(scores[i], comp_ids[i]))
    tup.sort(reverse=True)

    top = []
    visited = []
    for t in tup:
        if t[1] not in visited:
            top.append(t)
            visited.append(t[1])

    idss = [x[1] for x in top]
    if len(idss) > 20:
        idss = idss[:20]

    competency_names = []
    for ids in idss:
        competency_names.append(onet_df['category'].iloc[ids] + " | " + \
                                onet_df['competency'].iloc[ids] + " | " + \
                                onet_df['description'].iloc[ids])

    competency.append(competency_names)

In [49]:
result_df = occupations.iloc[job_ids[:FIRST_N]]

In [50]:
result_df['onet_competency'] = competency

/home/sumukha21/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
# Calculating the polarity of each reviews given by employees
result_df['polarity'] = result_df['description'].map(lambda text: TextBlob(text).sentiment.polarity)

polarities = result_df['polarity'].values.tolist()

reviews=[]

# Classifying the review as based on polarity score
for p in polarities:
    if p > 0.01 and p < 0.3:
        reviews.append('Good')
        
    elif p >= 0.3:
        reviews.append('Excellent')
        
    elif p < -0.01 and p > -0.3:
        reviews.append('Bad')
        
    elif p <= -0.3:
        reviews.append('Very Bad')
        
    else:
        reviews.append('Neutral')

result_df['reviews'] = reviews

/home/sumukha21/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/sumukha21/venv/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [52]:
result_df.head(10)

,id,title,identifier,score_all,name,description,onet_competency,polarity,reviews
0,1,Data Scientist,15-1111.00,0.883895,Computer and Information Research Scientists,Not a bad job but can get boring easily. Manag...,[Work Values | Recognition | Occupations that ...,0.154762,Good
1,2,Business Intelligence Analyst,15-1199.08,0.866399,Business Intelligence Analysts,co-workers are friendly but pay is not good at...,[Work Values | Relationships | Occupations tha...,0.151190,Good
2,3,Human Resources Data Scientist,15-1111.00,0.737225,Computer and Information Research Scientists,They give you the illusion of how great of a j...,[Work Values | Achievement | Occupations that ...,0.179167,Good
3,4,Lead - Human Resource Data Scientist,15-1111.00,0.738833,Computer and Information Research Scientists,"Basically, in BMO agile team, they hire 5+ per...",[Work Values | Working Conditions | Occupation...,0.000000,Neutral
4,5,Machine Learning Engineer,15-1111.00,0.847708,Computer and Information Research Scientists,Work/life balance and customer centric. Traini...,[Work Values | Working Conditions | Occupation...,0.250000,Good
5,6,Data Scientist,15-1111.00,0.846623,Computer and Information Research Scientists,A typical work day includes problem solving cu...,[Work Values | Working Conditions | Occupation...,0.281061,Good
6,7,Associate Data Scientist,15-1111.00,0.872069,Computer and Information Research Scientists,I am currently a student working part time at ...,[Work Values | Working Conditions | Occupation...,0.050000,Good
7,8,Data Scientist I,15-1111.00,0.884682,Computer and Information Research Scientists,everyone I so supportive and will work with yo...,[Work Values | Working Conditions | Occupation...,0.500000,Excellent
8,9,"Data Scientist, AI@Unity",15-1111.00,0.876431,Computer and Information Research Scientists,The best part of working at a company like BMO...,[Work Values | Working Conditions | Occupation...,0.750000,Excellent
9,10,Data Scientist,15-1111.00,0.847733,Computer and Information Research Scientists,BMO's culture is nothing like i've seen before...,[Work Values | Working Conditions | Occupation...,0.350000,Excellent


# Saving the Results

In [53]:
# Save file
result_df.to_csv('results/work_values.csv', index=False)